https://www.datacourses.com/extracting-data-from-gmail-emails-with-python-470/

https://codehandbook.org/how-to-read-email-from-gmail-api-using-python/

https://codehandbook.org/how-to-read-email-from-gmail-using-python/

In [57]:
UN  = "david92liu@gmail.com"
PW    = "andesmint"
url = "imap.gmail.com"
port   = 993

def readmail():
    # mail reading logic will come here !!
    return 0


In [58]:
mail = imaplib.IMAP4_SSL(url)
mail.login(UN,PW)

('OK', [b'david92liu@gmail.com authenticated (Success)'])

In [59]:
mail.select('inbox')

type, data = mail.search(None, 'ALL')
mail_ids = data[0]
id_list = mail_ids.split()


In [60]:
int(id_list[0])

1